In [ ]:
import datetime

import datajoint as dj

# Use the virtual modules
action = dj.create_virtual_module("action", "u19_action")
acquisition = dj.create_virtual_module("acquisition", "u19_acquisition")
behavior = dj.create_virtual_module("behavior", "u19_behavior")
lab = dj.create_virtual_module("lab", "u19_lab")
subject = dj.create_virtual_module("subject", "u19_subject")
puffs = dj.create_virtual_module("puffs", "u19_puffs")
scheduler = dj.create_virtual_module("scheduler", "u19_scheduler")
task = dj.create_virtual_module("task", "u19_task")

# This line calculates for yesterday in case you run this early in the morning before training
today = datetime.date.today() - datetime.timedelta(days=1)

[2025-03-26 09:59:39,182][INFO]: Connecting ct5868@datajoint00.pni.princeton.edu:3306
[2025-03-26 09:59:39,952][INFO]: Connected ct5868@datajoint00.pni.princeton.edu:3306


## Get the Subject's Most recent Status 

The following tables are inner joined and prepped for the future left joins

- subject.SubjectStatus
- subject.Subject
- subject.CagingStatus

### Info to Left Join

- Trained today
- Watered today
- Weighed today


In [2]:
# Get max effective_date per subject
max_dates = dj.U('subject_fullname').aggr(
    action.SubjectStatus,
    last_status='MAX(effective_date)'
)

sub_cag_status_exclusion = (action.SubjectStatus & max_dates.proj(effective_date='last_status')).proj('subject_status','water_per_day','schedule',unused_date ='effective_date') * subject.Subject() * subject.CagingStatus()
display(sub_cag_status_exclusion)

subject_fullname username_mouse_nickname,unused_date,subject_status,water_per_day in mL,schedule,subject_nickname,user_id username,genomics_id number from the facility,sex sex,dob birth date,head_plate_mark little drawing on the head plate for mouse identification,location,protocol protocol number,line name,subject_description description,initial_weight,notification_enabled,need_reweight,cage
ariordan_529,2021-01-01,Dead,nan,Nothing/Nothing/Nothing/Nothing/Nothing/Nothing/Nothing,529,ariordan,None,Male,2019-11-04,=BLOB=,valhalla,1910,Unknown,,33.9,1,0,(grave)
efonseca_EF002,2020-03-27,Dead,1.0,Nothing/Nothing/Nothing/Nothing/Nothing/Nothing/Nothing,EF002,efonseca,None,Male,2019-12-20,=BLOB=,valhalla,1910,Thy1-GP5.3 -/-,,23.0,1,0,(grave)
efonseca_EF003,2020-03-27,Dead,1.0,Nothing/Nothing/Nothing/Nothing/Nothing/Nothing/Nothing,EF003,efonseca,None,Male,2019-12-20,=BLOB=,vivarium,1910,Thy1-GP5.3 -/-,,22.2,1,0,grave
efonseca_EF004,2020-03-27,Dead,1.0,Nothing/Nothing/Nothing/Nothing/Nothing/Nothing/Nothing,EF004,efonseca,None,Male,2019-12-20,=BLOB=,vivarium,1910,Thy1-GP5.3 -/-,,21.8,1,0,grave
efonseca_ef111_act111,2024-08-18,Dead,1.0,Train/Train/Train/Train/Train/Train/Train,ef111_act111,efonseca,None,Female,None,=BLOB=,valhalla,1910,C57BL6/J,,21.7,1,0,(grave)
efonseca_ef112_act112,2024-01-29,Dead,1.0,Train/Train/Train/Train/Train/Train/Train,ef112_act112,efonseca,None,Female,None,=BLOB=,valhalla,1910,C57BL6/J,,20.3,1,0,(grave)
efonseca_ef113_act113,2024-08-18,Dead,1.0,Train/Train/Train/Train/Train/Train/Train,ef113_act113,efonseca,None,Female,None,=BLOB=,valhalla,1910,C57BL6/J,,21.8,1,0,(grave)
efonseca_ef114_act114,2024-08-18,Dead,1.0,Train/Train/Train/Train/Train/Train/Train,ef114_act114,efonseca,None,Female,None,=BLOB=,valhalla,1910,C57BL6/J,,21.0,1,0,(grave)
efonseca_ef150_actp004,2021-12-02,Dead,1.0,Nothing/Nothing/Nothing/Nothing/Nothing/Nothing/Nothing,ef150_actp004,efonseca,None,Male,2021-05-10,=BLOB=,valhalla,1910,Thy1-GP5.3,,30.4,1,0,(grave)
efonseca_ef158_actp005,2021-10-08,Dead,2.0,Water/Water/Water/Water/Water/Water/Water,ef158_actp005,efonseca,None,Female,2021-05-12,=BLOB=,valhalla,1910,Thy1-GP5.3,,23.6,1,0,(grave)


### Calculating the Training, Water Admin and Training

In [3]:
# Obtain if training occured today
today_training_sessions = dj.U('subject_fullname').aggr(
    acquisition.Session() & f'DATE(session_date) = "{today}"',
    last_session='MAX(session_date)'
)

todays_water_admin = (action.WaterAdministration & f'DATE(administration_date) = "{today}"').proj(
    "earned",
    "supplement",
    "received",
    "watertype_name",
    effective_date = 'administration_date')


# First, get the max weighing times for today
today_max_weighing_times = dj.U('subject_fullname').aggr(
    action.Weighing() & f'DATE(weighing_time) = "{today}"',  # Filter for today
    # last_weigh='MAX(weighing_time)',
    weigh='round(avg(weight),2)',
    effective_date = 'date(MAX(weighing_time))'
)


# Water and training. Let's add weighing
weight_water_training = todays_water_admin.join(today_training_sessions, left=True).join(
    today_max_weighing_times, left=True
)

weight_water_training_fetch = weight_water_training.fetch(format='frame')
weight_water_training_fetch

,,earned,supplement,received,watertype_name,last_session,weigh
subject_fullname,effective_date,,,,,,
efonseca_ef559_act121,2025-03-25,0.00000,0.0,0.00000,Unknown,None,None
efonseca_ef562_act122,2025-03-25,0.00000,0.0,0.00000,Unknown,None,None
efonseca_ef756_act123,2025-03-25,0.00000,0.0,0.00000,Unknown,None,None
efonseca_ef757_act124,2025-03-25,0.00000,0.0,0.00000,Unknown,None,None
jeremyjc_j064,2025-03-25,1.00590,0.0,1.00590,Unknown,2025-03-25,22.0
jeremyjc_j065,2025-03-25,0.00000,1.0,1.00000,Unknown,None,21.2
jeremyjc_j070,2025-03-25,0.00000,1.0,1.00000,Unknown,None,21.2
jeremyjc_j083,2025-03-25,1.58479,0.0,1.58479,Unknown,2025-03-25,22.9
jeremyjc_j084,2025-03-25,0.00000,1.0,1.00000,Unknown,None,23.9


In [4]:
all_together = (
    sub_cag_status_exclusion.join(weight_water_training, left=True)
    & 'subject_status not in ("Dead", "Missing", "AdLibWater")'
)
all_together_fetch = all_together.fetch(format='frame')
all_together_fetch

,,,subject_status,water_per_day,schedule,subject_nickname,user_id,genomics_id,sex,dob,head_plate_mark,location,...,initial_weight,notification_enabled,need_reweight,cage,earned,supplement,received,watertype_name,last_session,weigh
subject_fullname,unused_date,effective_date,,,,,,,,,,,,,,,,,,,,,
efonseca_ef559_act121,2025-03-20,2025-03-25,WaterRestrictionOnly,1.0,Train/Train/Train/Train/Train/Train/Train,ef559_act121,efonseca,None,Male,2024-02-13,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",vivarium,...,24.5,1,0,Active_Imaging1,0.0000,0.0,0.0000,Unknown,None,None
efonseca_ef562_act122,2025-02-04,2025-03-25,WaterRestrictionOnly,1.0,Water/Water/Water/Water/Water/Water/Water,ef562_act122,efonseca,None,Female,2024-02-13,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",vivarium,...,19.4,1,0,Active_Imaging2,0.0000,0.0,0.0000,Unknown,None,None
efonseca_ef756_act123,2025-02-04,2025-03-25,WaterRestrictionOnly,1.0,Water/Water/Water/Water/Water/Water/Water,ef756_act123,efonseca,None,Male,2024-04-17,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",vivarium,...,24.5,1,0,Active_Imaging3,0.0000,0.0,0.0000,Unknown,None,None
efonseca_ef757_act124,2025-02-04,2025-03-25,WaterRestrictionOnly,1.0,Water/Water/Water/Water/Water/Water/Water,ef757_act124,efonseca,None,Male,2024-04-17,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",vivarium,...,26.8,1,0,Active_Imaging3,0.0000,0.0,0.0000,Unknown,None,None
jeremyjc_j064,2024-05-17,2025-03-25,InExperiments,1.0,Train/Train/Train/Train/Train/Train/Train,j064,jeremyjc,None,Female,2024-03-03,None,vivarium,...,20.2,1,0,JC023,1.0059,0.0,1.0059,Unknown,2025-03-25,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
testuser_test100,2021-08-09,NaN,InExperiments,5.0,Train/Train/Train/Train/Train/Train/Train,test100,testuser,None,Male,2021-02-05,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",vivarium,...,25.0,1,0,Test5,NaN,NaN,NaN,None,None,None
testuser_test70,2021-08-06,NaN,InExperiments,50.0,Train/Train/Train/Train/Train/Train/Train,test70,testuser,None,Male,2021-02-05,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",vivarium,...,32.0,1,0,Test6,NaN,NaN,NaN,None,None,None
testuser_test80,2021-08-09,NaN,InExperiments,5.0,Train/Train/Train/Train/Train/Train/Train,test80,testuser,None,Male,2021-02-05,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",vivarium,...,25.0,1,0,Test5,NaN,NaN,NaN,None,None,None
